# Projet Machine Learning : Xente Fraud Detection
Auteurs : Théo Engels, Hadrien Godts

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
# import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.feature_selection import mutual_info_regression
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import OneHotEncoder
from xgboost import XGBRegressor
from sklearn.pipeline import Pipeline as skPipeline
from imblearn.pipeline import Pipeline as imbPipeline
from sklearn.compose import ColumnTransformer
from imblearn.over_sampling import SMOTE, SMOTENC
from CustomTransformers import StringCleanTransformer, DayTimeTransformer, DropperTransformer, SignTransformer, OHTransformer, FloatTransformer, biningTransformer, weekdayTransformer
from sklearn.model_selection import train_test_split

from xgboost import XGBClassifier

from sklearn.model_selection import StratifiedKFold

In [2]:
# chargement des données
train = pd.read_csv("data/training.csv")
test = pd.read_csv("data/test.csv")

# shuffle et indication de la target
train = train.sample(frac=1).reset_index(drop=True)
train_Y = train.FraudResult
train.drop(['FraudResult'], axis=1, inplace=True)

# features dont le string est à nettoyer
StringToClean = ["TransactionId", "BatchId","AccountId","SubscriptionId","CustomerId", "ProviderId", "ProductId", "ChannelId", "ProductCategory"]

# features à dropper, one-hot-encoder et binariser respectivement, et initialisation du smote
drop_cols = ["CurrencyCode", "BatchId", "CountryCode", "CustomerId", "PricingStrategy", "Amount"]
hot_cols = ["ProductCategory"]
bin_cols = ["TransactionStartTime"]
smt  = SMOTE()

## Introduction
Ce rapport présente les processus et les résultats de notre travail sur le Xente fraud detection challenge. Nous commençons par une analyse des données présentes dans le dataset, suivie de la hase de feature engineering. Nous présentons ensuite les différents modèles que nous avons entrainés et leurs performances, et nous terminons par une courte conclusion et quelques perspectives de recherche.

L’objectif de ce challenge est de prédire si une transaction bancaire est frauduleuse ou pas, sur la base des différents features contenus dans le dataset.


## Analyse des données

La première chose importante à noter est que le training set contient 95 662 entrées, parmi lesquelles on compte 95 469 transactions légales et 193 frauduleuses (soit 0.2 %) : c’est un déséquilibre fort et important à noter car le manque de transactions frauduleuses risque de compliquer l’entrainement des modèles. Nous pallions cette faiblesse dans le feature engineering.

In [3]:
# Code pour nbr transactions
print("Nb de transactions : ",train_Y.count())
print("Nb de fraudes : ",train_Y.value_counts()[1])
print("Nb de non fraudes : ",train_Y.value_counts()[0])
print("ratio fraudes/non frondes : ",train_Y.value_counts()[1] / train_Y.value_counts()[0])


Nb de transactions :  95662
Nb de fraudes :  193
Nb de non fraudes :  95469
ratio fraudes/non frondes :  0.0020215986341115964


La seconde chose importante à propos des données est qu’il ne manque aucune donnée : toutes les entrées ont des données pour chaque feature.

In [4]:
# Code pour données manquantes

print("Are there missing values : ", train.isnull().values.any())

train.head()

are there missing values :  False


,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy
0,TransactionId_11186,BatchId_11996,AccountId_4841,SubscriptionId_3829,CustomerId_1096,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-50.0,50,2019-01-31T07:49:33Z,2
1,TransactionId_89411,BatchId_39631,AccountId_3809,SubscriptionId_4505,CustomerId_4252,UGX,256,ProviderId_6,ProductId_14,financial_services,ChannelId_3,5250.0,5250,2019-02-07T21:48:56Z,2
2,TransactionId_73421,BatchId_73381,AccountId_4841,SubscriptionId_3829,CustomerId_2531,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-25.0,25,2019-02-05T05:24:15Z,2
3,TransactionId_80808,BatchId_94983,AccountId_4841,SubscriptionId_3829,CustomerId_2433,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-5000.0,5000,2018-12-21T09:25:54Z,2
4,TransactionId_131750,BatchId_109354,AccountId_3595,SubscriptionId_1749,CustomerId_4033,UGX,256,ProviderId_6,ProductId_4,airtime,ChannelId_3,1000.0,1000,2018-12-23T17:26:43Z,2


Voyons ensuite les différents feature du dataset. S’il y a des choses importantes, nous les mentionnons ici.
-	TransactionId : numéro de la transaction. Cette donnée est à garder car elle permet de soumettre les résultats au défi, mais nous retirerons cette colonne du dataset dans la mesure où le numéro de la transaction ne devrait rien nous apprendre de particulier sur son caractère frauduleux.
-	BatchId : le numéro de groupe de transaction. Comme pour le numéro de transaction, cette donnée ne devrait rien nous apprendre sur le caractère frauduleux d’une transaction. Par ailleurs, dans l’hypothèse improbable où toutes les transactions sont regroupées dans le même groupe, nous ferions face à du target leakage puisque le numéro de Batch nous dirait directement quelque chose à propos de la légalité de la transacition.
-	AccountId : l’identifiant d’un compte client. Nous conservons cette donnée.
-	SubscriptionId : l’identifiant d’une souscription. D’après la description des features, cette donnée semble liée à un compte client, sans autre explication particulièrement révélatrice. Vu ce lien et ce manque de clarté, nous décidons de laisser cette variable hors du dataset, afin d’en réduire la taille.
-	CustomerId : l’identifiant d’un client. À priori, un client peut disposer de plusieurs comptes, donc pour réduire la taille du dataset, nous laissons cette variable hors du dataset en privilégiant AccountId. Cela étant, si on peut établir qu’un compte fraude plus que les autres, il sera pertinent de reconstruire le lien vers le CustomerId.
-	CurrencyCode : la monnaie de la transaction. Il n’y a qu’une seule valeur pour ce feature, donc nous le laissons également hors du dataset.
-	CountryCode : le code du pays de la transaction. Idem que CurrencyCode, nous le laissons donc également hors du dataset.
-	ProviderId KEEP : indique la source du produit acheté via la transaction. Nous conservons cette feature en faisant un one-hot encoding sur les six valeurs possibles, en partant du principe qu'il peut y avoir un lien entre la source du produit et le caractère frauduleux de la transaction.
-	ProductId : donne le code du produit qui a été acheté lors de la transaction. Cette colonne contient 23 valeurs possibles et ne se prête donc pas à du one-hot encoding. En outre, nous disposons de la variable ProductCategory (voir ci-après) donc nous décidons de laisser celle-ci hors du dataset. 
-	ProductCategory : indique la catégorie de produit qui a été fait l’objet de la transaction. Il y a 9 catégories possibles, donc nous pouvons faire du one-hot encoding pour conserver cette feature.
-	ChannelId : indique la méthode de paiement utilisée pour effectuer la transaction. Cette feature nous semble importante, nous la conservons donc en faisant du one-hot encoding une fois de plus.
-	Amount : indique le montant de la transaction, avec son signe (+ ou - pour crédit ou débit). La feature value nous donne déjà le montant absolu, donc nous faisons donc d'abord du binary encoding pour que les montant positif soient désormais true, et les montant négatifs false.
-	Value : indique le montant de la transaction, sans son signe. Nous conservons cette feature.
-	TransactionStartTime : indique la date et l'horaire de la transaction. Nous convertissons le format de cette donnée pour avoir un ordre chronologique, puis nous la séparons en une colonne date et une colonne heure. La colonne date est convertie pour avoir le jour de la semaine et one-hot encodée ensuite. La colonne heure laissée telle quelle pour le moment.
-	PricingStrategy : indique la strucure de prix de Xente pour ses différents marchands. Sans certitude sur ce que cette colonne représente exactement, nous la one-hot encodons et verrons par la suite si elle a un impact ou non.
-	FraudResult : indique si la transaction est frauduleuse. Il s’agit de notre target, nous la gardons donc.

In [5]:
# Code pour nunique

print("nunique :\n", train.nunique())

nunique :
 TransactionId           95662
BatchId                 94809
AccountId                3633
SubscriptionId           3627
CustomerId               3742
CurrencyCode                1
CountryCode                 1
ProviderId                  6
ProductId                  23
ProductCategory             9
ChannelId                   4
Amount                   1676
Value                    1517
TransactionStartTime    94556
PricingStrategy             4
dtype: int64


## Feature engineering

Le dataset est déjà séparé en training et en validation, ce qui nous épargne cette étape, et nous l'avons déjà mélangé au moment de la lecture. Afin d’organiser efficacement notre travail, nous rédigeons le code sous forme de pipelines.

Nous créons un préprocesseur pour la transformation des données que nous appliquons au training set. Les transformations du premier préprocesseur sont décrites dans le tableau suivant :

| Transformateur | Description |
| --- | --- |
| clean | Pour les colonnes qui ont un id précédé d'un string (par exemple : BatchID_578), retire le texte en ne conserve que le numéro |
| amount_to_sign | Remplace les valeurs de la colonne amount par true si elles sont positives et false si elles sont négatives ; la colonne est renommée « sign ». |
| day_time_separator | Convertit le format de TransactionStartTime en format yyyymmddhhmmss, et sépare la colonne en jour (TransactionStartDay) et en heure (TransactionStartTime). |
| dropper | Supprime les features marqués comme non-pertinents (voir analyse des données) |
| one_hot_encode | Effectue un one-hot-encoding pour les colonnes suivantes : ProviderId, ProductCategory, ChannelId, PricingStrategy et Sign. |
| weekday | Convertit la colonne TransactionStartDay en jour de la semaine plutôt qu'en date  |
| float | Impose le type float sur toutes les colonnes (certaines colonnes du one-hot encoding ne sont plus en float). De plus, il passe les transactionId (dont nous avons besoin pour la soumission) en index. |
| smote | Applique un échantillonnage artificiel sur le dataset. |

Notez que le smote sera ignoré lors du predict(test)	

In [7]:
# Code pour le pipeline

pipeline = imbPipeline(steps = [
    ("clean", StringCleanTransformer()),
    ("amout to sign", SignTransformer()),
    ("day_time_separator", DayTimeTransformer()),
    ("Dropper", DropperTransformer(drop_cols)),
    ("One hot encoding", OHTransformer(hot_cols)),
    ("weekday", weekdayTransformer()),
    ("float", FloatTransformer()),
    ("smote", smt),
    ("model", XGBClassifier(n_estimators = 500))
])

## Modèles
Nous entrainons les données adaptées sur trois modèles : un Random Forest, un XGBoost et un MLP ; le tableau suivant montre les performances de ces modèles, sans adaptation particulière de leurs paramètres.


Notez que nous faisons initialement une validation croisée sur le training set, qui a tendance à donner des résultats incohérent avec les résultats donnés par le site. Nous ne nous basons donc exlusivement sur le site pour évaluer la qualité de nos modèles.

In [8]:
# Code pour le modèle de base

pipelineRF = imbPipeline(steps = [
    ("clean", StringCleanTransformer()),
    ("amout to sign", SignTransformer()),
    ("day_time_separator", DayTimeTransformer()),
    ("Dropper", DropperTransformer(drop_cols)),
    ("One hot encoding", OHTransformer(hot_cols)),
    ("weekday", weekdayTransformer()),
    ("float", FloatTransformer()),
    ("smote", smt),
    ("model", RandomForestClassifier())
])

pipelineXGB= imbPipeline(steps = [
    ("clean", StringCleanTransformer()),
    ("amout to sign", SignTransformer()),
    ("day_time_separator", DayTimeTransformer()),
    ("Dropper", DropperTransformer(drop_cols)),
    ("One hot encoding", OHTransformer(hot_cols)),
    ("weekday", weekdayTransformer()),
    ("float", FloatTransformer()),
    ("smote", smt),
    ("model", XGBClassifier())
])

pipelineMLP= imbPipeline(steps = [
    ("clean", StringCleanTransformer()),
    ("amout to sign", SignTransformer()),
    ("day_time_separator", DayTimeTransformer()),
    ("Dropper", DropperTransformer(drop_cols)),
    ("One hot encoding", OHTransformer(hot_cols)),
    ("weekday", weekdayTransformer()),
    ("float", FloatTransformer()),
    ("smote", smt),
    ("model", MLPClassifier())
])

if (False):
    for pipeline, name in zip([pipelineRF, pipelineXGB, pipelineMLP],["pipelineRF", "pipelineXGB", "pipelineMLP"]):
        # print("cross validation score : ", cross_val_score(pipeline, train, train_Y, cv=5, scoring="f1").mean())
        copy_train = train.copy()
        copy_train_Y = train_Y.copy()
        pipeline.fit(copy_train, copy_train_Y)
        test_res = pipeline.predict(test.copy())

        output = pd.DataFrame()
        output["TransactionId"] = test["TransactionId"]
        output["FraudResult"] = test_res

        #save the result to csv file
        output.to_csv("submission"+name+".csv", index=False)




In [9]:
def OH(X, hot_cols):
    OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
    for elem in hot_cols:
        OH_cols = pd.DataFrame(OH_encoder.fit_transform(X[elem].values.reshape(-1,1)))
        OH_cols.rename(columns=lambda x: elem + str(x), inplace=True)
        OH_cols.index = X.index
        X = pd.concat([X, OH_cols], axis=1)
    return X

drop_cols_diff = ["CurrencyCode", "BatchId", "CountryCode", "CustomerId", "PricingStrategy", "Amount", "ProductCategory"]

pipelineXGB_diff= imbPipeline(steps = [
    ("clean", StringCleanTransformer()),
    ("amout to sign", SignTransformer()),
    ("day_time_separator", DayTimeTransformer()),
    ("Dropper", DropperTransformer(drop_cols_diff)),
    # ("One hot encoding", OHTransformer(hot_cols)),
    ("weekday", weekdayTransformer()),
    ("float", FloatTransformer()),
    ("smote", smt),
    ("model", XGBClassifier())
])

print("cross validation score : ", cross_val_score(pipelineXGB_diff, OH(train.copy(), hot_cols), train_Y, cv=5, scoring="f1").mean())

pipelineXGB_diff.fit(OH(train.copy(), hot_cols), train_Y)
test_res = pipelineXGB_diff.predict(OH(test.copy(), hot_cols))

output = pd.DataFrame()
output["TransactionId"] = test["TransactionId"]
output["FraudResult"] = test_res

#save the result to csv file
output.to_csv("submission"+"diff_CV_zindi"+".csv", index=False)


cross validation score :  0.8836062933050884


Les résultats sont les suivants :

| Modèle | Public score | Private score |
| --- | --- | --- |
| Random Forest | 0.561403508 | 0.517241379 |
| XGBoost | 0.741935483 | 0.744186046 |
| MLP | 0.070323488 | 0.070412999 |

![Random Forest Result](./RFinitialScore.png)
![XGBoost Result](./XGBoostinitialScore.png)
![MLP Result](./MLPinitialScore.png)

Nous voyons clairement que le XGBoost présente les meilleurs résultats. Nous nous tentons donc de partir de cette base pour améliorer encore le modèle. Nous travaillons principalement sur les paramètres suivants  :
-	Le type de booster
-	n_estimators
-	learning rate
-	max_depth



In [10]:
# Code pour le modèle avec paramètres améliorés
pipelineXGB= imbPipeline(steps = [
    ("clean", StringCleanTransformer()),
    ("amout to sign", SignTransformer()),
    ("day_time_separator", DayTimeTransformer()),
    ("Dropper", DropperTransformer(drop_cols)),
    ("One hot encoding", OHTransformer(hot_cols)),
    ("weekday", weekdayTransformer()),
    ("float", FloatTransformer()),
    ("smote", smt),
    ("model", XGBClassifier(learning_rate = 0.5, max_depth = 4, n_estimators = 600))
])


Voici les résultats obtenus :

| Adaptation | Public score | Private score |
| --- | --- | --- |
| (fill) | (fill) | (fill) |
| (fill) | (fill) | (fill) |
| (fill) | (fill) | (fill) |


Par ailleurs, nous faisons les essais suivants :
-	Effectuer l’échantillonnage artificiel avant le feature engineering dans le training set
-	Conserver le feature « ProductId »

In [11]:
# Code pour le modèle final

## Conclusion et perspectives
[Conclusion]

Le score obtenu nous semble satisfaisant, d'autant plus qu'il nous place parmi les 128 meilleurs scores du concours, mais nous pensons qu’il est possible de l’améliorer encore. Nous avons plusieurs pistes pour y parvenir :
-	Faire du binning sur l'heure pour avoir matin - journée - soir au lieu de l'heure exacte.

-	L1 & L2
-	subsample
-	min_child_weight
-	gamma

-	Faire davantage de feature engineering avec davantage de domaine knowledge. Au regard des résultats obtenus par les participants au concours, il doit y avoir une manipulation de données qui permet d'améliorer nettement le score, mais à ce stade-ci nous manquons probablement de connaissances spécifiques pour la trouver.
